In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime

In [2]:
url = "https://www.worldometers.info/coronavirus/#nav-yesterday2"

In [3]:
def parse(url):    
    html_text = requests.get(url).text
    return BeautifulSoup(html_text, 'html.parser')

In [4]:
def get_columns(parser):
    # xóa các khoảng cách thừa
    columns = parser.find('thead').text.strip().replace('\n\n','\n')
    # thay kí tự unikey
    columns = columns.replace('\xa0', ' ')
    # xóa kí tự \n thừa
    columns = columns.replace('/\n','/').replace('\nContinent','')
    #
    columns = columns.replace('ppl', 'ppl\n')
    
    
    return columns.split('\n')[0:-1]

In [5]:
def raw_data(parser,yesterday= True, two_day_ago= False):
    if yesterday:
        parser1 = parser.find('table', {'id':'main_table_countries_yesterday'})
        raw1 = parser1.find_all('tr')[9:-8]
    if two_day_ago:
        parser2 = parser.find('table', {'id':'main_table_countries_yesterday2'})
        raw2= parser2.find_all('tr')[9:-8]
    if yesterday and two_day_ago:
        return (raw1,raw2)
    else:
        if yesterday:
            return raw1
        else:
            return raw2

In [6]:
def get_content(raw):
    start = False
    s = ""
    for ch in raw:
        if ch == '>':
            start = True
        elif ch == '<':
            if start == True: 
                start = False
                if s[-1] == '\n':
                    s = s[0:-1]
                else:
                    s = s + '\t'
        else:
            if start == True:
                if ch != ',':
                    s = s + ch
    index = [1,3,17,19,21,23]
    l1 = np.array(s.split('\t'))
    if len(l1) == 24:
        index = [1,3,17,19,21,23]
    else:
        index = [1,3,16,18,19,21,23,25]
    return list(np.delete(l1,index)) 

In [7]:
def save(df,filename):
    df.to_csv(filename)

In [14]:
def get_data(raw,columns):
    data= [get_content(str(i)) for i in raw]
    df = pd.DataFrame(data,columns= columns)
    df.set_index('#',inplace= True)
    num_cols = columns[2:]
    df[num_cols] = df[num_cols].astype('int64', errors='ignore')
    return df

In [15]:
def store_data(raw, columns, day):
    today = datetime.datetime.now()
    d= datetime.timedelta(days = day)
    filename = today - d
    day = today - d
    filename='Worldometer_' + day.strftime('%m_%d')[1:] + '.csv'
    df= get_data(raw,columns)
    save(df,filename)

In [10]:
parser = parse(url)

In [16]:
columns = get_columns(parser)

In [17]:
#raw1, raw2 = raw_data(parser,True,True)

In [18]:
#store_data(raw1, columns,1)
#store_data(raw2, columns,2)